In [21]:
import pandas as pd
import numpy as np
from sqlalchemy.orm import session
from sqlalchemy import create_engine

from datetime import datetime, timedelta
import pytz # new import

import gc

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

# Data preparation

In [22]:
data = pd.read_csv("fina_data.csv")

In [23]:
data.head()

,timestamp,device_name,current(A),power_factor,temperature(?),voltage(V)
0,07/04/2018 04:00,Lavaloza,0.0,NaN,12.2,NaN
1,07/04/2018 04:15,Lavaloza,0.0,NaN,12.2,NaN
2,07/04/2018 04:30,Lavaloza,0.0,NaN,12.2,NaN
3,07/04/2018 04:45,Lavaloza,0.0,NaN,12.2,NaN
4,10/04/2018 05:00,Lavaloza,0.0,NaN,12.2,NaN


In [24]:
data.rename(columns = {"temperature(?)":"temperature(°C)"}, inplace= True)

In [25]:
data.head()

,timestamp,device_name,current(A),power_factor,temperature(°C),voltage(V)
0,07/04/2018 04:00,Lavaloza,0.0,NaN,12.2,NaN
1,07/04/2018 04:15,Lavaloza,0.0,NaN,12.2,NaN
2,07/04/2018 04:30,Lavaloza,0.0,NaN,12.2,NaN
3,07/04/2018 04:45,Lavaloza,0.0,NaN,12.2,NaN
4,10/04/2018 05:00,Lavaloza,0.0,NaN,12.2,NaN


In [26]:
data["timestamp"] = pd.to_datetime(data["timestamp"])

In [27]:
data.iloc[0,0]

Timestamp('2018-07-04 04:00:00')

In [28]:
#Get other time data
data["hour"] = data["timestamp"].apply(lambda x: x.time().hour)
data["minute"] = data["timestamp"].apply(lambda x: x.time().minute)
data["time"] = data["hour"] + data["minute"]/60
data["weekday"] = data["timestamp"].apply(lambda x: x.weekday())
data["month"] = data["timestamp"].apply(lambda x: x.month)
data.head()

,timestamp,device_name,current(A),power_factor,temperature(°C),voltage(V),hour,minute,time,weekday,month
0,2018-07-04 04:00:00,Lavaloza,0.0,NaN,12.2,NaN,4,0,4.00,2,7
1,2018-07-04 04:15:00,Lavaloza,0.0,NaN,12.2,NaN,4,15,4.25,2,7
2,2018-07-04 04:30:00,Lavaloza,0.0,NaN,12.2,NaN,4,30,4.50,2,7
3,2018-07-04 04:45:00,Lavaloza,0.0,NaN,12.2,NaN,4,45,4.75,2,7
4,2018-10-04 05:00:00,Lavaloza,0.0,NaN,12.2,NaN,5,0,5.00,3,10


In [29]:
# Separate Main from devices
main_df = data[data["device_name"] == "Interruptor Principal"]
devices_df = data[data["device_name"] != "Interruptor Principal"]
del data
gc.collect()

7

In [30]:
main_df.head()

,timestamp,device_name,current(A),power_factor,temperature(°C),voltage(V),hour,minute,time,weekday,month
338340,2018-01-04 07:00:00,Interruptor Principal,36.43,0.95,12.0,127.0,7,0,7.00,3,1
338341,2018-01-04 07:15:00,Interruptor Principal,37.57,0.95,12.0,127.0,7,15,7.25,3,1
338342,2018-01-04 07:30:00,Interruptor Principal,31.58,0.95,12.0,127.0,7,30,7.50,3,1
338343,2018-01-04 07:45:00,Interruptor Principal,30.18,0.95,12.0,127.0,7,45,7.75,3,1
338344,2018-01-04 11:00:00,Interruptor Principal,117.65,0.95,15.9,127.0,11,0,11.00,3,1


In [31]:
devices_df.head()

,timestamp,device_name,current(A),power_factor,temperature(°C),voltage(V),hour,minute,time,weekday,month
0,2018-07-04 04:00:00,Lavaloza,0.0,NaN,12.2,NaN,4,0,4.00,2,7
1,2018-07-04 04:15:00,Lavaloza,0.0,NaN,12.2,NaN,4,15,4.25,2,7
2,2018-07-04 04:30:00,Lavaloza,0.0,NaN,12.2,NaN,4,30,4.50,2,7
3,2018-07-04 04:45:00,Lavaloza,0.0,NaN,12.2,NaN,4,45,4.75,2,7
4,2018-10-04 05:00:00,Lavaloza,0.0,NaN,12.2,NaN,5,0,5.00,3,10


In [32]:
devices_df.drop(columns = ["power_factor", "voltage(V)", "hour","time", "weekday", "month", "temperature(°C)","minute"], inplace =True)

In [33]:
devices_df.head()

,timestamp,device_name,current(A)
0,2018-07-04 04:00:00,Lavaloza,0.0
1,2018-07-04 04:15:00,Lavaloza,0.0
2,2018-07-04 04:30:00,Lavaloza,0.0
3,2018-07-04 04:45:00,Lavaloza,0.0
4,2018-10-04 05:00:00,Lavaloza,0.0


In [36]:
devices_df = pd.pivot_table(devices_df, index= "timestamp", columns = "device_name", values = "current(A)",
                            aggfunc="mean")\
                            .reset_index().sort_values(by = "timestamp", ascending =True).fillna(0)

In [37]:
devices_df.head()

device_name,timestamp,Lavaloza,Tablero A,Tablero B,Tablero C,Tablero D,Tablero E
0,2018-01-04 00:00:00,0.0,11.50,34.37,2.19,1.79,1.76
1,2018-01-04 00:15:00,0.0,3.47,26.89,2.27,1.77,1.80
2,2018-01-04 00:30:00,0.0,0.00,33.70,2.08,1.19,1.72
3,2018-01-04 00:45:00,0.0,0.00,28.57,3.75,1.48,1.70
4,2018-01-04 01:00:00,0.0,0.00,30.33,2.87,1.45,1.69


In [38]:
main_df.head()

,timestamp,device_name,current(A),power_factor,temperature(°C),voltage(V),hour,minute,time,weekday,month
338340,2018-01-04 07:00:00,Interruptor Principal,36.43,0.95,12.0,127.0,7,0,7.00,3,1
338341,2018-01-04 07:15:00,Interruptor Principal,37.57,0.95,12.0,127.0,7,15,7.25,3,1
338342,2018-01-04 07:30:00,Interruptor Principal,31.58,0.95,12.0,127.0,7,30,7.50,3,1
338343,2018-01-04 07:45:00,Interruptor Principal,30.18,0.95,12.0,127.0,7,45,7.75,3,1
338344,2018-01-04 11:00:00,Interruptor Principal,117.65,0.95,15.9,127.0,11,0,11.00,3,1


# Building the model

In [39]:
devices_to_train = ["Lavaloza", "Tablero A","Tablero B", "Tablero C", "Tablero D", "Tablero E"]
features = list(main_df.columns[2:])

In [40]:
features

['current(A)',
 'power_factor',
 'temperature(°C)',
 'voltage(V)',
 'hour',
 'minute',
 'time',
 'weekday',
 'month']

In [42]:
model_df = pd.merge(left = main_df, right = devices_df, on="timestamp")
del devices_df, main_df
gc.collect()

115

In [43]:
X = model_df[features]
y = model_df[devices_to_train]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Gradient Boosting Regressor

In [59]:
param_grid = {'n_estimators': [50, 60,70, 80, 100],
              'max_depth': [5,6,7],
              'learning_rate':[0.01, 0.1 , 0.3]}
grid = GridSearchCV(GradientBoostingRegressor(), param_grid, verbose=2, n_jobs = -1)

In [60]:
model_best_params = []

In [61]:
# Lavaloza
grid.fit(X_train_scaled, y_train["Lavaloza"].values.reshape(-1,1))
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  2.9min finished


The best parameters to use are:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}


In [62]:
model_best_params.append(grid.best_params_)

In [63]:
# Tablero A
grid.fit(X_train_scaled, y_train["Tablero A"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.1min finished


The best parameters to use are:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}


In [64]:
# Tablero B
grid.fit(X_train_scaled, y_train["Tablero B"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.1min finished


The best parameters to use are:  {'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 100}


In [65]:
# Tablero C
grid.fit(X_train_scaled, y_train["Tablero C"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.1min finished


The best parameters to use are:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}


In [66]:
# Tablero D
grid.fit(X_train_scaled, y_train["Tablero D"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.0min finished


The best parameters to use are:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}


In [67]:
#Tablero E
grid.fit(X_train_scaled, y_train["Tablero E"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.0min finished


The best parameters to use are:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}


In [75]:
model_best_params

[{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100},
 {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100},
 {'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 100},
 {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100},
 {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100},
 {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}]

In [69]:
model_list_gbr= []
counter = 0
for column in devices_to_train:
    y = y_train[column].values.reshape(-1,1)
    gbr = GradientBoostingRegressor(max_depth = model_best_params[counter]["max_depth"],
                                    n_estimators = model_best_params[counter]["n_estimators"],
                                    learning_rate= model_best_params[counter]["learning_rate"]).fit(X_train_scaled, y)
    model_list_gbr.append(gbr)
    counter += 1
    print("\nModel for {}".format(column))
    print(gbr.score(X_train_scaled, y_train[column].values.reshape(-1,1)))
    print(gbr.score(X_test_scaled, y_test[column].values.reshape(-1,1)))


Model for Lavaloza
0.867300175461742
0.7906378857478108

Model for Tablero A
0.9764653068581749
0.9660979786445633

Model for Tablero B
0.8035740224345015
0.6302551767115341

Model for Tablero C
0.7538957923425903
0.6124463033744582

Model for Tablero D
0.802243252322689
0.7009134201337076

Model for Tablero E
0.9668880926139838
0.948679855198246


In [74]:
corr_df = model_df[["Tablero A", "Tablero B","Tablero C", "Tablero D", "Tablero E", "current(A)"]]
corr_df.corr()

,Tablero A,Tablero B,Tablero C,Tablero D,Tablero E,current(A)
Tablero A,1.000000,-0.013605,0.603723,0.449698,0.752307,0.817439
Tablero B,-0.013605,1.000000,0.045520,0.132123,0.044135,0.156328
Tablero C,0.603723,0.045520,1.000000,0.297510,0.493204,0.596429
Tablero D,0.449698,0.132123,0.297510,1.000000,0.612794,0.623004
Tablero E,0.752307,0.044135,0.493204,0.612794,1.000000,0.878084
current(A),0.817439,0.156328,0.596429,0.623004,0.878084,1.000000


## Multilayer Perceptrons

In [79]:
param_grid = {'activation': ["relu", "tanh"],
              'hidden_layer_sizes': [[50, 50, 50, 50]]
             }
grid = GridSearchCV(MLPRegressor(), param_grid, verbose=3, n_jobs = -1)

In [80]:
model_best_params = []

In [81]:
# Lavaloza
grid.fit(X_train_scaled, y_train["Lavaloza"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.6min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.9min finished


The best parameters to use are:  {'activation': 'tanh', 'hidden_layer_sizes': [50, 50, 50, 50]}


In [ ]:
# Tablero A
grid.fit(X_train_scaled, y_train["Tablero A"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

In [82]:
# Tablero B
grid.fit(X_train_scaled, y_train["Tablero B"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  2.3min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.8min finished


The best parameters to use are:  {'activation': 'tanh', 'hidden_layer_sizes': [50, 50, 50, 50]}


In [83]:
# Tablero C
grid.fit(X_train_scaled, y_train["Tablero C"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  2.1min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.5min finished


The best parameters to use are:  {'activation': 'tanh', 'hidden_layer_sizes': [50, 50, 50, 50]}


In [84]:
# Tablero D
grid.fit(X_train_scaled, y_train["Tablero D"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  2.3min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.8min finished


The best parameters to use are:  {'activation': 'tanh', 'hidden_layer_sizes': [50, 50, 50, 50]}


In [85]:
#Tablero E
grid.fit(X_train_scaled, y_train["Tablero E"].values.reshape(-1,1))
model_best_params.append(grid.best_params_)
print('The best parameters to use are: ', grid.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.5min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.9min finished


The best parameters to use are:  {'activation': 'tanh', 'hidden_layer_sizes': [50, 50, 50, 50]}


In [88]:
model_list_mlp= []
counter = 0
for column in y_train.columns:
    y = y_train[column].values.reshape(-1,1)
    mlp = MLPRegressor(activation = model_best_params[counter]["activation"],
                                    hidden_layer_sizes = model_best_params[counter]["hidden_layer_sizes"]).fit(X_train_scaled, y)
    model_list_mlp.append(mlp)
    counter += 1
    print("Model for {}".format(column))
    print(mlp.score(X_train_scaled, y_train[column].values.reshape(-1,1)))
    print(mlp.score(X_test_scaled, y_test[column].values.reshape(-1,1)))
    print("\n")

Model for Lavaloza
0.7883067094969667
0.7664395962940823


Model for Tablero A
0.9642324040067558
0.9609542357023863


Model for Tablero B
0.5833091563503268
0.5637300499647405


Model for Tablero C
0.6651404427827086
0.5836310334252517


Model for Tablero D
0.7185879975921012
0.6738848719114111




IndexError: list index out of range

In [92]:
y = y_train["Tablero E"].values.reshape(-1,1)
mlp = MLPRegressor(activation = model_best_params[5]["activation"],
                                    hidden_layer_sizes = model_best_params[counter]["hidden_layer_sizes"]).fit(X_train_scaled, y)

# Create CSV

In [95]:
features_predict = scaler.transform(model_df[features])

for device, model_mlp, model_gbr in zip(devices_to_train, model_list_mlp, model_list_gbr):
    model_df[device + "_pred_mlp"] = model_mlp.predict(features_predict)
    model_df[device + "_pred_gbr"] = model_gbr.predict(features_predict)
model_df

,timestamp,device_name,current(A),power_factor,temperature(°C),voltage(V),hour,minute,time,weekday,...,Tablero A_pred_mlp,Tablero A_pred_gbr,Tablero B_pred_mlp,Tablero B_pred_gbr,Tablero C_pred_mlp,Tablero C_pred_gbr,Tablero D_pred_mlp,Tablero D_pred_gbr,Tablero E_pred_mlp,Tablero E_pred_gbr
0,2018-01-04 07:00:00,Interruptor Principal,36.43,0.95,12.00,127.0,7,0,7.00,3,...,3.330976,1.013106,21.509257,19.672065,3.047945,3.576994,1.809254,1.954966,1.732249,1.755064
1,2018-01-04 07:15:00,Interruptor Principal,37.57,0.95,12.00,127.0,7,15,7.25,3,...,4.371762,0.985848,19.707644,19.672065,3.523957,3.929764,1.810041,1.954966,1.444563,1.609371
2,2018-01-04 07:30:00,Interruptor Principal,31.58,0.95,12.00,127.0,7,30,7.50,3,...,3.002654,0.800496,18.946326,19.799574,3.117753,2.864594,1.812792,1.913154,1.484630,1.897600
3,2018-01-04 07:45:00,Interruptor Principal,30.18,0.95,12.00,127.0,7,45,7.75,3,...,3.262910,0.976128,17.668379,19.601560,3.526848,2.471907,1.803346,1.866544,1.439598,1.902580
4,2018-01-04 11:00:00,Interruptor Principal,117.65,0.95,15.90,127.0,11,0,11.00,3,...,7.375878,8.149896,17.355874,16.424779,5.801446,4.821766,1.803647,2.966638,22.868988,33.890868
5,2018-01-04 11:15:00,Interruptor Principal,121.59,0.95,15.90,127.0,11,15,11.25,3,...,7.569564,8.242419,16.942450,16.275541,5.834322,4.864887,2.184155,2.966638,25.535364,40.003514
6,2018-01-04 11:30:00,Interruptor Principal,119.75,0.95,15.90,127.0,11,30,11.50,3,...,7.599322,8.279079,18.098484,15.499571,5.848631,4.864887,2.322822,2.969067,25.125342,36.017845
7,2018-01-04 11:45:00,Interruptor Principal,166.50,0.95,15.90,127.0,11,45,11.75,3,...,8.221876,7.924438,16.122137,13.154416,6.039793,4.964497,2.924555,6.201649,49.435613,52.926100
8,2018-01-04 12:00:00,Interruptor Principal,164.78,0.95,18.00,127.0,12,0,12.00,3,...,8.846422,8.679521,15.711536,13.267620,5.512211,5.333176,4.872987,5.776840,49.201059,55.803566
9,2018-01-04 12:15:00,Interruptor Principal,177.37,0.95,18.00,127.0,12,15,12.25,3,...,9.341663,9.304358,16.620681,15.420167,5.475685,5.498644,5.940335,9.552259,52.078856,57.090311


In [96]:
model_df.to_csv("predicciones.csv")

In [ ]:
model_list_mlp

In [ ]:
features